In [ ]:
%pip install selenium-driverless

# Installing Chrome on GitHub Actions (Ubuntu)
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!sudo apt install ./google-chrome-stable_current_amd64.deb -y

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="got execution_context_id and unique_context=True, defaulting to execution_context_id")

import os
import time
import random
import asyncio
from datetime import datetime, timedelta, timezone

from selenium_driverless import webdriver
from selenium_driverless.types.by import By

# email_prefix = "hsiaoseanhs"
# bir = 911119
acc = os.environ['ANKI_ACCOUNT']
pas = os.environ['ANKI_PASSWORD']
remain = -1

In [ ]:
test = False

In [ ]:
class Selenium:
    def __init__(self):
        self.driver = None

    async def async__init__(self):
        options = webdriver.ChromeOptions()
        options.add_argument("--mute-audio")
        options.add_argument("--headless=new")  # Run in headless mode for GitHub Actions
        options.add_argument("--no-proxy-server")
        options.add_argument("--disable-gpu")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-features=NetworkService")
        options.add_argument("--no-sandbox")  # Added for GitHub Actions
        
        options.add_argument("--disable-blink-features=AutomationControlled")
        
        # Detect environment: GitHub Actions (Ubuntu) vs local (macOS)
        # import platform
        # if platform.system() == "Linux":  # GitHub Actions
        options.binary_location = "/usr/bin/google-chrome"
        chrome_dir = "/tmp/Chrome"
        # else:  # Local macOS
        #     options.binary_location = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
        #     chrome_dir = "/Users/hsiao/Chrome"
            
        # Create Chrome profile directory
        os.makedirs(chrome_dir, exist_ok=True)
        
        # Set user data directory
        options.add_argument(f"--user-data-dir={chrome_dir}")

        self.driver = await webdriver.Chrome(options=options)
        
        # await self.driver.delete_all_cookies()
        await self.driver.execute_cdp_cmd("Emulation.setTimezoneOverride", {"timezoneId": "Asia/Taipei"})
        await self.driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {"source": """Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"""})

    async def screenshot(self, filename="screenshot.png"):
        await self.driver.save_screenshot(filename)

    async def login(self):
        while True:
            try: 
                await self.driver.get("https://ankiweb.net/account/login", wait_load=True)
                Login = await self.driver.find_element(By.CLASS_NAME, "btn btn-primary btn-lg", timeout=1)
                svelte = await self.driver.find_elements(By.CLASS_NAME, "form-control svelte-1ak1s42")
                await svelte[0].send_keys(acc)
                await svelte[1].send_keys(pas)
                await Login.click()
                break
            except: 
                await self.driver.sleep(1)

    async def practice(self, target):
        while True:
            try:
                element = await self.driver.find_element(By.XPATH, "//*[text()='(02)English']", timeout=1)
                await element.click()
                break
            except: await self.driver.sleep(1)

        count = target
        while count > 0:
            try:
                element = await self.driver.find_element(By.XPATH, "//*[text()='Show Answer']", timeout=1)
                await element.click()
                await self.driver.sleep(random.randint(50,60))
                while True:
                    try:
                        btn = await self.driver.find_elements(By.CLASS_NAME, "btn btn-primary btn-lg m-1", timeout=1)
                        if len(btn) == 4: 
                            remain = await self.driver.find_element(By.CSS_SELECTOR, 'div.float-end')
                            remain = await remain.text
                            remain = remain.replace("\n", "")
                            goodtime = await self.driver.find_element(By.XPATH, '//*[@id="ansarea"]/div/div[3]/div')
                            goodtime = await goodtime.text
                            if goodtime == "10m":
                                await btn[2].click()
                                print(f"G 10m({remain}){count}")
                            elif goodtime == "01d":
                                await btn[2].click()
                                print(f"G 01d({remain}){count}")
                            else:
                                R = random.random()
                                if R <= 0.3:
                                    await btn[2].click()
                                    print(f"G ran({remain}){count}")
                                elif R <= 0.6:
                                    await btn[1].click()
                                    print(f"H ran({remain}){count}")
                                else:
                                    await btn[0].click()
                                    print(f"A ran({remain}){count}")
                            count -= 1
                            break
                    except: await self.driver.sleep(1)
            except: await self.driver.sleep(1)
        await self.driver.quit()

In [ ]:
if test:
    random_Q = 5
else:
    random_Q = random.randint(20, 60)
    
print(f"Starting at {datetime.now(timezone(timedelta(hours=8))).strftime('%Y-%m-%d %H:%M:%S')}")
print(f"{random_Q} cards")

sln = Selenium()
try:
    await sln.async__init__()
    await sln.login()
    await sln.practice(random_Q)
except Exception as e:
    print(f"Error running AnkiWeb automation: {e}")
finally:
    print(f"Finished at {datetime.now(timezone(timedelta(hours=8))).strftime('%Y-%m-%d %H:%M:%S')}")